# Model Bias in NLP - Application to Hate Speech

# Classification

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, GRU, Dropout,Embedding ,Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
# Download Kaggle Dataset via the Kaggle API

kaggle_api = {"username":"bojonas","key":"1b3903e6a0f6c0bb4c684ade1e291321"}

import json
with open('/content/kaggle.json', 'w') as file:
    json.dump(kaggle_api, file)

!chmod 600 /content/kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

!unzip \*.zip  && rm *.zip

 96% 265M/276M [00:03<00:00, 139MB/s]
100% 276M/276M [00:03<00:00, 89.5MB/s]
 69% 11.0M/15.9M [00:00<00:00, 19.0MB/s]
100% 15.9M/15.9M [00:00<00:00, 45.5MB/s]
 70% 11.0M/15.8M [00:00<00:00, 25.9MB/s]
100% 15.8M/15.8M [00:00<00:00, 35.3MB/s]
 58% 7.00M/12.1M [00:00<00:00, 71.0MB/s]
100% 12.1M/12.1M [00:00<00:00, 77.5MB/s]
 77% 50.0M/64.7M [00:00<00:00, 81.3MB/s]
100% 64.7M/64.7M [00:00<00:00, 110MB/s] 
 97% 315M/326M [00:03<00:00, 95.6MB/s]
100% 326M/326M [00:03<00:00, 96.1MB/s]
 90% 11.0M/12.3M [00:00<00:00, 43.7MB/s]
100% 12.3M/12.3M [00:00<00:00, 48.6MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 66.6MB/s]
Archive:  test_public_expanded.csv.zip
  inflating: test_public_expanded.csv  

Archive:  identity_individual_annotations.csv.zip
  inflating: identity_individual_annotations.csv  

Archive:  all_data.csv.zip
  inflating: all_data.csv            

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  sample_submission.csv.zip
  inflating: sampl

In [ ]:
# Check TPU Device

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.57.231.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.231.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [ ]:
# Read training data

train = pd.read_csv("/content/train.csv")

X = train["comment_text"]
Y = train["target"]

In [ ]:
# Tokenize to 10'000 words and pad to 100 words per comment

tokenizer = Tokenizer(num_words=10000, oov_token="UNK")
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, 100)

In [ ]:
# Train Test split for training/testing

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# Define Keras LSTM Model with embedding layer

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=50, input_length=100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           500000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               91648     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 591,777
Trainable params: 591,777
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Getting TPU Adresses

try:
 device_name = os.environ["COLAB_TPU_ADDR"]
 TPU_ADDRESS = "grpc://" + device_name
 print("Found TPU at: {}".format(TPU_ADDRESS))
except KeyError:
 print("TPU not found")

Found TPU at: grpc://10.57.231.42:8470


In [ ]:
opt = tf.optimizers.Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
train_dataset_X = tf.data.Dataset.from_tensors(X_train)
train_dataset_Y = tf.data.Dataset.from_tensors(Y_train)

In [ ]:
model.fit(X_train,Y_train,batch_size=64, epochs=7, validation_data=(X_test, Y_test))

Epoch 1/7
22561/22561 [==============================] - 2767s 122ms/step - loss: 0.2482 - acc: 0.7008 - val_loss: 0.2443 - val_acc: 0.7007
Epoch 2/7
11900/22561 [==============>...............] - ETA: 19:55 - loss: 0.2423 - acc: 0.7009